# 1. Problem Statement

- We use Union All clause to combine Sales data and their Goals measures in the same query to look at the data side by side.
- In the Union queries, we use NULL as a placeholder for the columns that don't have equivalent values in its respective positional location.

Example:

SELECT SUPPLIER, BRAND, GROUPS, SALES_AMT, NULL GROUP_GOAL FROM SALES

UNION ALL

SELECT NULL, NULL, GROUPS, NULL, GROUP_GOAL FROM GROUPS_GOALS

- We want the output of SALES records along with the Goals data side by side as columns. And, we don't want to show the GOALES records separately in the output.
- To achieve this, we do group by on all dimensional columns and aggregate the metrics columns. 
- In addition to this, we want to rollup the data to every hierarchy level using GROUPING_SET. 
- This GROUPING_SET also applies NULL values to the dimensional columns when it rollsup the data. As a result, the dimensional columns will be duplicated as they also have NULL values due to the UNION queries we had at first place.

#### How do we fix the duplicate issue in the resultset?

#### Below code is to setup the css configuration for better formatting of our SQL output

In [34]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [35]:
import duckdb

### 2. Creating a Table at Supplier, Brand, Family, Group and Category levels with Sales Metrics. Inserting 4 records.

In [36]:
duckdb.sql ("create or replace table sales (supplier varchar, brand varchar, family varchar, groups varchar, \
             category varchar, sales numeric(10,2))")
duckdb.sql ("insert into sales values \
            ('Abc', 'Apple', 'Touch', '5G', 'Smart Phone', 100), \
            ('Abc', 'Asus', 'Touch', '4G', 'Smart Phone', 150), \
            ('Xyz', 'Samsung', 'Flip', '5G', 'Tablet', 200), \
            ('Xyz', 'Redmi', 'Touch', '4G', 'Smart Phone', 250)")

Creating a Table at Supplier level with Goals defined. Inserting 2 records.

In [41]:
duckdb.sql ("create or replace table supplier (supplier varchar, supplier_goals numeric (10,2))")
duckdb.sql ("insert into supplier values ('Abc', 44444), ('Xyz', 55555)")

Creating a Table at Supplier & Brand Level with goals defined. Inserting 4 records.

In [42]:
duckdb.sql ("create or replace table brand (supplier varchar, brand varchar, brand_goals numeric (10,2))")
duckdb.sql ("insert into brand values ('Abc', 'Apple', 77777), ('Xyz', 'Samsung', 55111), \
            ('Abc', 'Asus', 22999), ('Xyz', 'Redmi', 11999)")

Creating a Table at Supplier, Brand & Family level with Goals defined. Inserting 4 records.

In [43]:
duckdb.sql ("create or replace table family (supplier varchar, brand varchar, family varchar, family_goals numeric (10,2))")
duckdb.sql ("insert into family values ('Abc', 'Apple', 'Touch', 66666), ('Xyz', 'Samsung', 'Flip', 33333), \
            ('Abc', 'Asus', 'Touch', 22222), ('Xyz', 'Redmi', 'Touch', 11111)")

Creating a Table at Group going across all other goals tables, with Goals defined. Inserting 2 records.

In [44]:
duckdb.sql ("create or replace table groups (groups varchar, group_goals numeric (10,2))")
duckdb.sql ("insert into groups values ('4G', 44444), ('5G', 88888)")

Creating a Table at category going across all other goals tables, with Goals defined. Inserting 2 records.

In [45]:
duckdb.sql ("create or replace table category (category varchar, cat_goals numeric (10,2))")
duckdb.sql ("insert into category values ('Smart Phone', 99999), ('Tablet', 33777)")

### 3. Showing data from alll the tables

In [46]:
duckdb.sql ("from sales").show()
duckdb.sql ("from supplier").show()
duckdb.sql ("from brand").show()
duckdb.sql ("from family").show()
duckdb.sql ("from groups").show()
duckdb.sql ("from category").show()

┌──────────┬─────────┬─────────┬─────────┬─────────────┬───────────────┐
│ supplier │  brand  │ family  │ groups  │  category   │     sales     │
│ varchar  │ varchar │ varchar │ varchar │   varchar   │ decimal(10,2) │
├──────────┼─────────┼─────────┼─────────┼─────────────┼───────────────┤
│ Abc      │ Apple   │ Touch   │ 5G      │ Smart Phone │        100.00 │
│ Abc      │ Asus    │ Touch   │ 4G      │ Smart Phone │        150.00 │
│ Xyz      │ Samsung │ Flip    │ 5G      │ Tablet      │        200.00 │
│ Xyz      │ Redmi   │ Touch   │ 4G      │ Smart Phone │        250.00 │
└──────────┴─────────┴─────────┴─────────┴─────────────┴───────────────┘

┌──────────┬────────────────┐
│ supplier │ supplier_goals │
│ varchar  │ decimal(10,2)  │
├──────────┼────────────────┤
│ Abc      │       44444.00 │
│ Xyz      │       55555.00 │
└──────────┴────────────────┘

┌──────────┬─────────┬───────────────┐
│ supplier │  brand  │  brand_goals  │
│ varchar  │ varchar │ decimal(10,2) │
├──────────┼──

### 4. Union all tables augmenting the columns to each other. I have given NULL for dimensional columns in the UNION queries other than first query

In [47]:
duckdb.sql ("create or replace table union1 as \
             select supplier, brand, family, groups, category, sales, null supplier_goals, null brand_goals, \
             null family_goals, null group_goals, null cat_goals from sales \
             union all \
             select supplier, NULL brand, NULL as family, NULL as groups, NULL as category, NULL as sales, supplier_goals, \
             NULL as brand_goals, NULL family_goals, null group_goals, null cat_goals from supplier \
             union all \
             select supplier, brand, NULL as family, NULL as groups, NULL as category, NULL as sales, NULL as supplier_goals, \
             brand_goals, NULL family_goals, null group_goals, null cat_goals from brand \
             union all \
             select supplier, brand, family, NULL as groups, NULL as category, NULL as sales, null as supplier_goals, \
             NULL as brand_goals, family_goals, null group_goals, null cat_goals from family \
             union all \
             select null as supplier, null as brand, null as family, groups, NULL as category, NULL as sales, \
             null as supplier_goals, NULL as brand_goals, null as family_goals, group_goals, null cat_goals from groups \
             union all \
             select null as supplier, null as brand, null as family, NULL as groups, category, NULL as sales, \
             null as supplier_goals, NULL as brand_goals, null as family_goals, \
             null as group_goals, cat_goals from category")

In [48]:
duckdb.sql ("from union1").show(max_rows=100)

┌──────────┬─────────┬─────────┬─────────┬─────────────┬───────────────┬────────────────┬───────────────┬───────────────┬───────────────┬───────────────┐
│ supplier │  brand  │ family  │ groups  │  category   │     sales     │ supplier_goals │  brand_goals  │ family_goals  │  group_goals  │   cat_goals   │
│ varchar  │ varchar │ varchar │ varchar │   varchar   │ decimal(10,2) │ decimal(10,2)  │ decimal(10,2) │ decimal(10,2) │ decimal(10,2) │ decimal(10,2) │
├──────────┼─────────┼─────────┼─────────┼─────────────┼───────────────┼────────────────┼───────────────┼───────────────┼───────────────┼───────────────┤
│ Abc      │ Apple   │ Touch   │ 5G      │ Smart Phone │        100.00 │           NULL │          NULL │          NULL │          NULL │          NULL │
│ Abc      │ Asus    │ Touch   │ 4G      │ Smart Phone │        150.00 │           NULL │          NULL │          NULL │          NULL │          NULL │
│ Xyz      │ Samsung │ Flip    │ 5G      │ Tablet      │        200.00 │    

### 5. Applying GROUPING_SET clause to get the rollup data at Supplier, Brand, Family, Groups and Category levels and overall report level.

### Showing the Problem

In [65]:
duckdb.sql ("select row_number () over (order by supplier, brand, family, groups, category) \
             as row_num, \
             supplier, brand, family, groups, category, sum(sales) sales, \
             sum (supplier_goals) supplier_goals, sum (brand_goals) brand_goals, sum (family_goals) family_goals, \
             sum (group_goals) group_goals, sum (cat_goals) cat_goals, \
             grouping_id (supplier, brand, family, groups, category) as grouping_id \
             from union1 \
             group by grouping sets ((supplier, brand, family, groups, category), \
             (supplier, brand, family), (supplier, brand), supplier, groups, category, ())").show(max_rows=100)

┌─────────┬──────────┬─────────┬─────────┬─────────┬─────────────┬───────────────┬────────────────┬───────────────┬───────────────┬───────────────┬───────────────┬─────────────┐
│ row_num │ supplier │  brand  │ family  │ groups  │  category   │     sales     │ supplier_goals │  brand_goals  │ family_goals  │  group_goals  │   cat_goals   │ grouping_id │
│  int64  │ varchar  │ varchar │ varchar │ varchar │   varchar   │ decimal(38,2) │ decimal(38,2)  │ decimal(38,2) │ decimal(38,2) │ decimal(38,2) │ decimal(38,2) │    int64    │
├─────────┼──────────┼─────────┼─────────┼─────────┼─────────────┼───────────────┼────────────────┼───────────────┼───────────────┼───────────────┼───────────────┼─────────────┤
│       1 │ Abc      │ Apple   │ Touch   │ 5G      │ Smart Phone │        100.00 │           NULL │          NULL │          NULL │          NULL │          NULL │           0 │
│       2 │ Abc      │ Apple   │ Touch   │ NULL    │ NULL        │          NULL │           NULL │          N

### In the above result set, we see the records 2 & 3, they have same dimensional columns ('Abc', 'Apple', 'Touch', Null, Null). This is due to one actual record from the union query and one grouping_set record.

### Likewise, there are many duplicate records in the above result set

### 6.Solution Approach

#### Using ENUM in the UNION query:

We introduce an ENUM column of boolean 0 (for the sales query) and 1 (all other queries) as query_number for each query in the UNION. This query_number column is used to filter the duplicate records at the end. We apply an aggregate function (Minimum) on this query_number column, and apply predicate query_number = 0. So we this will filter out:

1. All records from the second query onwards as they have the ENUM valuse as 1
2. Grouping sets duplicates as the minimum of 0 & 1 is 0

In [50]:
duckdb.sql ("create or replace table union2 as \
            select 0::boolean as union_query_num, supplier, brand, family, groups, category, sales, null supplier_goals, \
            null brand_goals, null family_goals, null group_goals, null cat_goals from sales \
            union \
            select 1, supplier, NULL brand, NULL as family, NULL as groups, NULL as category, NULL as sales, supplier_goals, \
            NULL as brand_goals, NULL family_goals, null group_goals, null cat_goals from supplier \
            union \
            select 1, supplier, brand, NULL as family, NULL as groups, NULL as category, NULL as sales, NULL as supplier_goals, \
            brand_goals, NULL family_goals, null group_goals, null cat_goals from brand \
            union \
            select 1, supplier, brand, family, NULL as groups, NULL as category, NULL as sales, null as supplier_goals, \
            NULL as brand_goals, family_goals, null group_goals, null cat_goals from family \
            union \
            select 1, null as supplier, null as brand, null as family, groups, NULL as category, NULL as sales, \
            null as supplier_goals, NULL as brand_goals, null as family_goals, group_goals, null cat_goals from groups \
            union \
            select 1, null as supplier, null as brand, null as family, NULL as groups, category, NULL as sales, \
            null as supplier_goals, NULL as brand_goals, null as family_goals, null as group_goals, cat_goals from category")

In [51]:
duckdb.sql ("from union2")

┌─────────────────┬──────────┬─────────┬─────────┬─────────┬─────────────┬───────────────┬────────────────┬───────────────┬───────────────┬───────────────┬───────────────┐
│ union_query_num │ supplier │  brand  │ family  │ groups  │  category   │     sales     │ supplier_goals │  brand_goals  │ family_goals  │  group_goals  │   cat_goals   │
│      int32      │ varchar  │ varchar │ varchar │ varchar │   varchar   │ decimal(10,2) │ decimal(10,2)  │ decimal(10,2) │ decimal(10,2) │ decimal(10,2) │ decimal(10,2) │
├─────────────────┼──────────┼─────────┼─────────┼─────────┼─────────────┼───────────────┼────────────────┼───────────────┼───────────────┼───────────────┼───────────────┤
│               1 │ NULL     │ NULL    │ NULL    │ 5G      │ NULL        │          NULL │           NULL │          NULL │          NULL │      88888.00 │          NULL │
│               1 │ Xyz      │ Samsung │ Flip    │ NULL    │ NULL        │          NULL │           NULL │          NULL │      33333.00 │ 

### 7. Applying the GROUPING SETS to the UNION result. At the end of the following query, Filtering the union_query_number = 0 will result only getting sales records from UNION result and removes the duplicates derived through GROUPING SET records.

In [52]:
duckdb.sql ("with group_query_3 as \
            (select supplier, brand, family, groups, category, min(union_query_num) leaf_level, \
            sum(sales) sales, sum (supplier_goals) supplier_goals, sum (brand_goals) brand_goals, \
            sum (family_goals) family_goals, sum (group_goals) group_goals, sum (cat_goals) cat_goals, \
            grouping_id (supplier, brand, family, groups, category) level1 \
            from union2 \
            group by grouping sets ((supplier, brand, family, groups, category), (supplier, brand, family), \
            (supplier, brand), supplier, groups, category, ())) \
            select supplier, brand, family, groups, category, sales, supplier_goals, brand_goals, family_goals, \
            group_goals, cat_goals, level1 \
            from group_query_3 where leaf_level = 0 \
            order by ifnull(supplier, '0000'), ifnull (brand, '0000'), ifnull (family, '0000'), \
            ifnull (groups, '0000'), ifnull (category, '0000')").show(max_rows=100)

┌──────────┬─────────┬─────────┬─────────┬─────────────┬───────────────┬────────────────┬───────────────┬───────────────┬───────────────┬───────────────┬────────┐
│ supplier │  brand  │ family  │ groups  │  category   │     sales     │ supplier_goals │  brand_goals  │ family_goals  │  group_goals  │   cat_goals   │ level1 │
│ varchar  │ varchar │ varchar │ varchar │   varchar   │ decimal(38,2) │ decimal(38,2)  │ decimal(38,2) │ decimal(38,2) │ decimal(38,2) │ decimal(38,2) │ int64  │
├──────────┼─────────┼─────────┼─────────┼─────────────┼───────────────┼────────────────┼───────────────┼───────────────┼───────────────┼───────────────┼────────┤
│ NULL     │ NULL    │ NULL    │ NULL    │ NULL        │        700.00 │       99999.00 │     167886.00 │     133332.00 │     133332.00 │     133776.00 │     31 │
│ NULL     │ NULL    │ NULL    │ NULL    │ Smart Phone │        500.00 │           NULL │          NULL │          NULL │          NULL │      99999.00 │     30 │
│ NULL     │ NULL    │